In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [1]:
# Importing required libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Loading the movie data
!ls dataset/

genome-scores.csv  links.csv  movies.csv   README.txt
genome-tags.csv    ml-latest  ratings.csv  tags.csv


In [3]:
# Reading the data

df_movies = pd.read_csv('dataset/movies.csv')
df_links = pd.read_csv('dataset/links.csv')
df_ratings = pd.read_csv('dataset/ratings.csv')
df_genome_tags = pd.read_csv('dataset/genome-tags.csv')
df_genome_scores = pd.read_csv('dataset/genome-scores.csv')

#merging scores and tags.
df_movie_tags_in_text = pd.merge(df_genome_scores, df_genome_tags, on='tagId')[['movieId', 'tag', 'relevance']]

# keep tags with relevnce higher than 0.3
df_movie_tags = df_genome_scores[df_genome_scores.relevance > 0.3][['movieId', 'tagId']]

In [4]:
# Display first movie in df_movies dataframe.
df_movies[df_movies.movieId == 1]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [11]:
# Looking at some of the tags for the above movie (Toy Story)

df_movie_tags[df_movie_tags['movieId'] == 1].merge(df_genome_tags, on= 'tagId').sample(10)

,movieId,tagId,tag
155,1,961,spying
105,1,600,life
27,1,204,children
172,1,1040,transformation
89,1,496,heartbreaking
182,1,1104,weird
72,1,417,funny
132,1,776,pg
179,1,1091,visually appealing
98,1,535,imagination


In [2]:
# Encoding Features Using the tf-idf(text frequency-inverse document frequency)

In [5]:

df_tags_to_movies = pd.merge(df_movie_tags, df_genome_tags, on='tagId', how='left')[['movieId', 'tagId']]
df_tags_to_movies['tagId'] = df_tags_to_movies.tagId.astype(str)

In [7]:
# Utility function to concateneate tags as one long string
def concatenate_movie_tags(tags):
    str_tags = ' '.join(set(tags))
    return str_tags

In [8]:
df_tags_per_movie = df_tags_to_movies.groupby('movieId')['tagId'].agg(concatenate_movie_tags)
df_tags_per_movie.name = 'movie_tags'
df_tags_per_movie = df_tags_per_movie.reset_index()

In [9]:
df_tags_per_movie[df_tags_per_movie['movieId'] == 1]

,movieId,movie_tags
0,1,138 536 719 969 128 364 49 497 415 1040 334 45...


In [10]:
df_avg_ratings  = df_ratings.groupby('movieId')['rating'].agg(['mean', 'median', 'size'])
df_avg_ratings.columns = ['rating_mean', 'rating_median', 'num_ratingsdf_tags_per_movie']
df_avg_ratings = df_avg_ratings.reset_index()

In [11]:
df_movies_with_ratings = pd.merge(df_movies, df_avg_ratings, how='left', on='movieId')

In [12]:
df_data = pd.merge(df_movies_with_ratings, df_tags_per_movie, how='left', on='movieId')

In [13]:
df_data_with_tags = df_data[~df_data.movie_tags.isnull()].reset_index(drop=True)

In [14]:
##TF_IDF vectors

In [16]:
tf_idf = TfidfVectorizer()

In [17]:
df_movies_tf_idf_described = tf_idf.fit_transform(df_data_with_tags.movie_tags)

In [18]:
m2m = cosine_similarity(df_movies_tf_idf_described)

: 

: 

In [ ]:
df_tfidf_m2m = pd.DataFrame(cosine_similarity(df_movies_tf_idf_described))